### Main params

In [13]:
remote_path = 'http://pacha.datawheel.us/economia/nene/occupational_status/'

local_path = '../data/'

engine_path = 'postgresql://localhost:5433/datachile'

### Imports

In [8]:
from urllib import request
import zipfile
import shutil
import os.path

import json
import pandas as pd
from sqlalchemy import create_engine

### Open file function

In [9]:
def loadFile(file_name):
    remote_file = remote_path + file_name
    local_file = local_path + file_name

    if not os.path.isfile(local_file):
        with request.urlopen(remote_file) as remote_csv,open(local_file, 'wb') as local_csv:
            shutil.copyfileobj(remote_csv, local_csv)
    
    return pd.read_csv(local_file,delimiter=",")

### Load file

In [10]:
df = loadFile('occupational_status.csv')
list(df)

['ano_encuesta',
 'mes_encuesta',
 'region_id',
 'age',
 'age_range_id',
 'sex_id',
 'icse_id',
 'isco_id',
 'isced_id',
 'occupied_id',
 'general_economic_condition',
 'fact']

### Complete NaN

In [11]:
df['isco_id'] = df['isco_id'].fillna('0')
df['icse_id'] = df['icse_id'].fillna('0')
df['isced_id'] = df['isced_id'].fillna('0')

### Rename, to int & load

In [14]:
df = df.rename(columns={'ano_encuesta':'year','mes_encuesta':'month'});
df = df.astype({'year':'int','month':'int','region_id':'int','age':'int','age_range_id':'int','icse_id':'int','sex_id':'int','isced_id':'int','isco_id':'int','occupied_id':'int','icse_id':'int','general_economic_condition':'int'})

engine = create_engine(engine_path)
df.to_sql('fact_occupational_status_nene', engine, schema='economy', if_exists='replace', index=False)

### Indexes & FK

In [15]:
engine.execute("""
ALTER TABLE economy.fact_occupational_status_nene
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE economy.fact_occupational_status_nene
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = economy.fact_occupational_status_nene.year
      AND dim_date.month_of_year = economy.fact_occupational_status_nene.month
      AND dim_date.day_of_month = 1
""")

engine.execute("""
CREATE INDEX fact_occupational_status_nene_region_id 
ON economy.fact_occupational_status_nene (region_id)
""")

In [52]:
def inline_table_xml(df, alias, id_column_name, desc_column_name):
    xml = """
<InlineTable alias="%(alias)s">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
  %(rows)s
  </Rows>
</InlineTable>
    """
    
    keys = [id_column_name, desc_column_name, desc_column_name]
    cols = ['id', 'description', 'es_description']
    
    
    rows = [("  <Row>\n%s\n    </Row>" % ("\n".join(["      <Value column=\"%s\">%s</Value>" % (cols[i], r[k]) for i, k in enumerate(keys)]))) for r in df.to_dict('records')]
    
    return xml % { 'alias': alias, 'rows': "\n".join(rows)}
    

### Related dim

In [54]:
remote_path = 'http://pacha.datawheel.us/ids_oficiales/'
d1 = loadFile('cine_isced.csv')
print (inline_table_xml(d1, 'isced', 'isced_id', 'isced'))


<InlineTable alias="isced">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Doctorado</Value>
      <Value column="es_description">Doctorado</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Educación Preescolar</Value>
      <Value column="es_description">Educación Preescolar</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Educación Primaria (nivel 1)</Value>
      <Value column="es_description">Educación Primaria (nivel 1)</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Educación Primaria (nivel 2)</Value>
      <Value column="es_description">Educación Primaria (nivel 2)</Value>
    </Row>
  <Row>
      <Value column="id">5</Val

In [55]:
d2 = loadFile('cise_icse.csv')
print (inline_table_xml(d2, 'icse', 'icse_id', 'icse'))


<InlineTable alias="icse">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Asalariado Sector Privado</Value>
      <Value column="es_description">Asalariado Sector Privado</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Asalariado Sector Público</Value>
      <Value column="es_description">Asalariado Sector Público</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Cuenta Propia</Value>
      <Value column="es_description">Cuenta Propia</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Empleador</Value>
      <Value column="es_description">Empleador</Value>
    </Row>
  <Row>
      <Value column="id">5</Value>
      <Value column="de

In [56]:
d3 = loadFile('ciuo_isco.csv')
print (inline_table_xml(d3, 'isco', 'isco_id', 'isco'))


<InlineTable alias="isco">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Agricultores y trabajadores calificados agropecuarios y pesqueros</Value>
      <Value column="es_description">Agricultores y trabajadores calificados agropecuarios y pesqueros</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Empleados de oficina</Value>
      <Value column="es_description">Empleados de oficina</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Funcionarios públicos y personal directivo de la administración pública y empresas</Value>
      <Value column="es_description">Funcionarios públicos y personal directivo de la administración pública y empresas</Value>
    </Row>
  <Row>
      <Valu

In [57]:
d4 = loadFile('age_ranges.csv')
print (inline_table_xml(d4, 'age_ranges', 'age_range_id', 'age_range'))


<InlineTable alias="age_ranges">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">15 a 19</Value>
      <Value column="es_description">15 a 19</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">20 a 24</Value>
      <Value column="es_description">20 a 24</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">25 a 29</Value>
      <Value column="es_description">25 a 29</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">30 a 65</Value>
      <Value column="es_description">30 a 65</Value>
    </Row>
  <Row>
      <Value column="id">5</Value>
      <Value column="description">65 a maxima edad de encuestados</Value>
      <Value column="es_descrip

In [16]:
d5 = loadFile('sex.csv')
d5

,sex_id,sex
0,0,Hombre
1,1,Mujer


In [58]:
d6 = loadFile('occupational_situation.csv')
print (inline_table_xml(d6, 'occupational_situation', 'occupied_id', 'occupied'))


<InlineTable alias="occupational_situation">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Ocupado</Value>
      <Value column="es_description">Ocupado</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Desocupado</Value>
      <Value column="es_description">Desocupado</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Inactivo</Value>
      <Value column="es_description">Inactivo</Value>
    </Row>
  </Rows>
</InlineTable>
    


In [59]:
d7 = loadFile('general_economic_condition.csv')
print (inline_table_xml(d7, 'general_economic_condition', 'general_economic_condition_id', 'general_economic_condition'))


<InlineTable alias="general_economic_condition">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">0</Value>
      <Value column="description">Menor de 15 años</Value>
      <Value column="es_description">Menor de 15 años</Value>
    </Row>
  <Row>
      <Value column="id">1</Value>
      <Value column="description">Ocupado tradicional</Value>
      <Value column="es_description">Ocupado tradicional</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Ocupado notradicional</Value>
      <Value column="es_description">Ocupado notradicional</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Ocupado ausente</Value>
      <Value column="es_description">Ocupado ausente</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
   